In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px 
import plotly.graph_objects as go 
import matplotlib.pyplot as plt
import seaborn as sns
#add number of loans issued over time
#average interest rate vs expected default rate with colours indicating subgrades
import plotly.io as pio
pio.templates.default = "presentation"
px_colour = px.colors.qualitative.Pastel
import kaleido

Classification - Default Ind definitely easiest, potentially loan grade classification
Regression - ?
NLP - ? 


In [ ]:
df = pd.read_csv("XYZCorp_LendingData.txt", sep = "\t", low_memory = False)

In [ ]:
df.shape

In [ ]:
df_ordered = df.reindex(sorted(df.columns), axis=1)

with pd.option_context('display.max_rows', 5, 'display.max_columns', None): 
    display(df_ordered)

In [ ]:
df.tail()

In [ ]:
df.select_dtypes("object")

In [ ]:
df.select_dtypes(["object"]).columns

In [ ]:
df["annual_inc_joint"].count()*100/len(df)

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
east = [
'CT',
'MA',
'ME',
'NH',
'NJ',
'NY',
'PA',
'RI',
'VT'
]
midwest = [
'IA',
'IL',
'IN',
'KS',
'MI',
'MN',
'MO',
'ND',
'NE',
'OH',
'SD',
'WI'
]
south = [
'AL',
'AR',
'DC',
'DE',
'FL',
'GA',
'KY',
'LA',
'MD',
'MS',
'NC',
'OK',
'SC',
'TN',
'TX',
'VA',
'WV'
]
west = [
'AK',
'AZ',
'CA',
'CO',
'HI',
'ID',
'MT',
'NM',
'NV',
'OR',
'UT',
'WA',
'WY'
]

In [ ]:
df_region = df.copy()

df_region["region"] = df_region["addr_state"].apply(lambda x: "East" if x in east else
                                       ("Midwest" if x in midwest else
                                        ("South" if x in south else
                                         ("West" if x in west else np.nan))))
limit = df["annual_inc"].quantile(0.99)

fig = px.histogram(df_region[df_region["annual_inc"]<limit], x = "annual_inc", color = "region", nbins = 30,
                  color_discrete_sequence = px_colour)
fig.update_layout(bargap=0.30,
                 title = "Distribution of Annual Incomes split by Region",
                 xaxis_title = "Annual Income (USD)",
                 yaxis_title = "Count",
                 legend_title = "US Region")
fig.write_image("annual_inc_region_distr.png")

In [ ]:


fig = px.histogram(df_region, x = "loan_amnt", color = "home_ownership", nbins = 30,
                  color_discrete_sequence = px.colors.qualitative.T10)
fig.update_layout(bargap=0.30,
                 title = "Distribution of Loan Amounts split by Home Ownership",
                 xaxis_title = "Loan Amount (USD)",
                 yaxis_title = "Count",
                 legend_title = "Home Ownership")
fig.write_image("loan_amnt_ownership_disr.png")

In [ ]:

fig = px.histogram(df_region[df_region["annual_inc"]<limit], x = "annual_inc", color = "region", nbins = 30, facet_col = "term",
                  color_discrete_sequence = px_colour,
                  labels = {"term":"Term "}, facet_col_spacing = 0.04)
fig.update_layout(bargap=0.30)
fig.update_layout(bargap=0.30,
                 title = "Distribution of Annual Incomes split by Term and Region",
                 xaxis_title = "Annual Income (USD)",
                 yaxis_title = "Count",
                 legend_title = "US Region")
fig.write_image("annual_inc_region_distr_term.png")

In [ ]:
limit = df["annual_inc"].quantile(0.99)

fig = px.histogram(df[df["annual_inc"]<limit], x = "annual_inc", color = "home_ownership", nbins = 30, facet_col = "term",
                  color_discrete_sequence = px_colour,
                  labels = {"term":"Term ", "annual_inc":"Annual Income (USD)"}, facet_col_spacing = 0.05)
fig.update_layout(bargap=0.30,
                 title = "Distribution of Annual Incomes split by Term and Home Ownership",
                 xaxis_title = "Annual Income (USD)",
                 yaxis_title = "Count",
                 legend_title = "Home Ownership", width = 1000, height = 600)

fig.write_image("annual_inc_ownership_distr_term.png")
fig.show()

In [ ]:
limit = df["annual_inc"].quantile(0.99)

fig = px.histogram(df[(df["annual_inc"]<limit) & (df["default_ind"] == 0)], x = "annual_inc", color = "home_ownership", nbins = 30, color_discrete_sequence = px_colour)
fig.update_layout(bargap=0.30,
                 title = "Distribution of Annual Incomes split by Home Ownership for Non-Defaulting Loans",
                 xaxis_title = "Annual Income (USD)",
                 yaxis_title = "Count",
                 legend_title = "Home Ownership")
fig.write_image("annual_inc_distr_non_default.png")
fig.show()

In [ ]:
limit = df["annual_inc"].quantile(0.99)

fig = px.histogram(df[(df["annual_inc"]<limit) & (df["default_ind"] == 1)], x = "annual_inc", color = "home_ownership", nbins = 30, color_discrete_sequence = px_colour)
fig.update_layout(bargap=0.30,
                 title = "Distribution of Annual Incomes split by Home Ownership for Defaulting Loans",
                 xaxis_title = "Annual Income (USD)",
                 yaxis_title = "Count",
                 legend_title = "Home Ownership")
fig.write_image("annual_inc_distr_non_default.png")
fig.show()

In [ ]:
df["application_type"].value_counts(normalize = True)

In [ ]:
df["default_ind"].value_counts(normalize = True)

In [ ]:
df["emp_title"]

In [ ]:
px.histogram(df, x = "int_rate")

In [ ]:
import datetime as dt
df["issue_d"] = pd.to_datetime(df["issue_d"])
df["issue_y"] = df["issue_d"].dt.year
df["term"] = df["term"].replace({"36 months":"36 Months","60 months":"60 Months"})

In [ ]:
import kaleido
fig = px.histogram(df, x = "issue_y", color = "term", color_discrete_sequence = px.colors.qualitative.T10, labels = {"36 months":"36 Months","60 months":"60 Months"})
fig.update_layout(bargap=0.30,
                 title = "Distribution of Loans Issued over Time split by Term",
                 xaxis_title = "Year",
                 yaxis_title = "Count",
                 legend_title = "Term")

fig.show()
fig.write_image("distr_loans_over_time.png")

In [ ]:
fig = px.histogram(df, x = "issue_y", color = "term")


In [ ]:
fig = make_subplots(rows = 1, cols = 2, subplot_titles = ["Loan Amount","Interest Rate"])
trace_amnt = go.Box(
        y  = df["loan_amnt"],
        x = df["term"].replace({"36 months":"36 Months","60 months":"60 Months"}),
        name = "Loan Amount"
)
trace_int = go.Box(
        y  = df["int_rate"],
        x = df["term"].replace({"36 months":"36 Months","60 months":"60 Months"}),
        name = "Interest Rate"
)
fig.append_trace(trace_amnt, 1, 1)
fig.append_trace(trace_int, 1, 2)
fig.update_layout(
                 title = "Loan Amount and Interest Rate split by Loan Term", width = 1000, height = 600)
fig.update_yaxes(title_text = "Loan Amount (USD)", row = 1, col =1)
fig.update_yaxes(title_text = "Interest Rate (%)", row = 1, col =2)
fig.update_xaxes(title_text = "Term")
fig.update_layout(legend=dict(orientation = "h",y = -0.2,x=0.24))

fig.write_image("box_plot_loan_amnt_interest.png")
fig.show()

In [ ]:
df["loan_amnt"].describe()

In [ ]:
df["title"]

In [ ]:
df["verification_status"].value_counts(normalize = True)

In [ ]:
df["grade"].value_counts()

In [ ]:
fig = px.histogram(df, x = "home_ownership")
fig.update_layout(bargap=0.30)


In [ ]:
fig=px.box(df, x = "grade", y = "int_rate", color = "grade", color_discrete_sequence = px_colour, category_orders={"grade":["A","B","C","D","E","F","G"]})
fig.update_xaxes(categoryorder='array', categoryarray= ["A","B","C","D","E","F","G"])
fig.update_layout(
                 title = "Distribution of Interest Rate split by Loan Grade",
                 yaxis_title = "Interest Rate (%)",
                 xaxis_title = "Loan Grade",
                 legend_title = "Loan Grade")
fig.write_image("int_rate_grade_boxplot.png")
fig.show()

In [ ]:
def test(series):
    return series.value_counts(normalize = True)

In [ ]:
grade_group = df.groupby(["grade"])[["default_ind"]].value_counts(normalize = True).reset_index()
grade_group_default = grade_group[grade_group["default_ind"] == 1]
fig = px.line(grade_group_default, x = "grade", y = 0, markers = True)

fig.update_layout(
                 title = "Probablility of Default vs Loan Grade",
                 yaxis_title = "Probability of Default",
                 xaxis_title = "Loan Grade")
fig.write_image("prob_default_vs_grade.png")
fig.show()

In [ ]:
inq_group = df.groupby(["inq_last_6mths"])[["default_ind"]].value_counts(normalize = True).reset_index()
inq_group_default = inq_group[inq_group["default_ind"] == 1]
fig = px.line(inq_group_default, x = "inq_last_6mths", y = 0, markers = True)

fig.update_layout(
                 title = "Probablility of Default vs No. Loan Inquiries over Previous 6 Months ",
                 yaxis_title = "Probability of Default",
                 xaxis_title = "No. Loan Inquiries over Previous 6 Months")
fig.write_image("prob_default_vs_no_inq.png")

In [ ]:
emp_length_group = df.groupby(["emp_length"])[["default_ind"]].value_counts(normalize = True).reset_index()
emp_length_group_default = emp_length_group[emp_length_group["default_ind"] == 1]
emp_length_group_default = emp_length_group_default.set_index("emp_length").reindex(index  = ["< 1 year","1 year","2 years","3 years","4 years","5 years","6 years","7 years","8 years","9 years","10+ years"]).reset_index()
px.line(emp_length_group_default, x = "emp_length", y = 0, markers = True, 
        category_orders = {"emp_length":["< 1 year","1 year","2 years","3 years","4 years","5 years","6 years","7 years","8 years","9 years","10+ years"]})

In [ ]:
df_cred = df.copy()
df_cred["earliest_cr_line"] = pd.to_datetime(df_cred["earliest_cr_line"]).dt.year
counts = df_cred["earliest_cr_line"].value_counts().to_dict()
df_cred["counts"] = df_cred["earliest_cr_line"].map(counts)
df_cred = df_cred[df_cred["counts"] >= 50]

In [ ]:
import datetime as dt
credit_group = df_cred.groupby(["earliest_cr_line"])[["default_ind"]].value_counts(normalize = True).reset_index()
credit_group_default = credit_group[credit_group["default_ind"] == 1]
px.scatter(credit_group_default, x = "earliest_cr_line", y = 0)


In [ ]:
credit_group_default[0].quantile(0.3333)

In [ ]:
[x for x in df.columns if "collection" in x]

default vs non-default percentage for:
collection vs no collection
derog record vs no derog record
delinq vs no delinq
payment plan vs no payment plan
36 months vs 60 months term
late fee vs no late fee

In [ ]:
df['total_rec_late_fee']

In [ ]:
df.columns

In [ ]:
df_db = df.copy()
df_db["collection_check"] = df_db['collections_12_mths_ex_med'].apply(lambda x: 1 if x > 0 else 0)
df_db["derog_check"] = ~df_db['mths_since_last_major_derog'].isna()
df_db["payment_check"] = ~df_db['mths_since_last_delinq'].isna()
df_db["pub_check"] = df_db['pub_rec'].apply(lambda x: 1 if x > 0 else 0)
df_db["late_check"] = df_db['total_rec_late_fee'].apply(lambda x: 1 if x > 0 else 0)
checks = ["collection_check","derog_check","payment_check","pub_check","late_check"]
true_perc = []
false_perc =[]
true_count, false_count = [],[]
for check in checks:
    true_perc.append(df_db[df_db[check] == 1]["default_ind"].value_counts()[1])
    true_count.append(len(df_db[df_db[check] == 1]["default_ind"]))
    false_perc.append(df_db[df_db[check] == 0]["default_ind"].value_counts()[1])
    false_count.append(len(df_db[df_db[check] == 0]["default_ind"]))

df_db_res = pd.DataFrame({"Check":checks,"True":true_perc,"True Count":true_count,"False":false_perc,"False Count":false_count})



In [ ]:
df_db = df.copy()
df_db["collection_check"] = df_db['collections_12_mths_ex_med'].apply(lambda x: 1 if x > 0 else 0)
df_db["derog_check"] = ~df_db['mths_since_last_major_derog'].isna()
df_db["payment_check"] = ~df_db['mths_since_last_delinq'].isna()
df_db["pub_check"] = df_db['pub_rec'].apply(lambda x: 1 if x > 0 else 0)
df_db["late_check"] = df_db['total_rec_late_fee'].apply(lambda x: 1 if x > 0 else 0)
checks = ["collection_check","derog_check","payment_check","pub_check","late_check"]
true_perc = []
false_perc =[]
true_count, false_count, true_not_def, false_not_def = [],[],[],[]
for check in checks:
    true_perc.append(df_db[df_db[check] == 1]["default_ind"].value_counts(normalize = True)[1]*100)
    true_not_def.append(df_db[df_db[check] == 1]["default_ind"].value_counts(normalize = True)[0]*100)
    true_count.append(len(df_db[df_db[check] == 1]["default_ind"]))
    false_perc.append(df_db[df_db[check] == 0]["default_ind"].value_counts(normalize = True)[1]*100)
    false_not_def.append(df_db[df_db[check] == 0]["default_ind"].value_counts(normalize = True)[0]*100)
    false_count.append(len(df_db[df_db[check] == 0]["default_ind"]))

df_db_res = pd.DataFrame({"Check":checks,"True - Defaulted":true_perc,"True Count":true_count,
                          "False - Defaulted":false_perc,"False Count":false_count,
                          "True - Didn't Default":true_not_def,"False - Didn't Default":false_not_def})



In [ ]:
df_db_res

In [ ]:
df_db_plot_true = df_db_res[["Check","True - Defaulted","True - Didn't Default"]].melt(id_vars = ["Check"])
df_db_plot_false = df_db_res[["Check","False - Defaulted","False - Didn't Default"]].melt(id_vars = ["Check"])
df_db_plot_false = df_db_plot_false.sort_values("Check")
df_db_plot_true = df_db_plot_true.sort_values("Check")

In [ ]:
from plotly.subplots import make_subplots
fig_true = make_subplots(rows=3, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}],[{'type':'domain'}, {'type':'domain'}],[{'type':'domain'}, {'type':'domain'}]],
                   subplot_titles = ["Collection Payment in Last 12 Months"," No Collection Payment in Last 12 Months","Borrower Delinquency","No Borrower Delinquency","Loan Payments Missed","No Loan Payments Missed"])
`
fig.add_trace(go.Pie(labels = df_db_plot_true[df_db_plot_true["Check"] == checks[0]]["variable"], values = df_db_plot_true[df_db_plot_true["Check"] == checks[0]]["value"]),1,1 )
fig.add_trace(go.Pie(labels = df_db_plot_false[df_db_plot_false["Check"] == checks[0]]["variable"], values = df_db_plot_false[df_db_plot_false["Check"] == checks[0]]["value"]),1,2 )
fig.add_trace(go.Pie(labels = df_db_plot_true[df_db_plot_true["Check"] == checks[2]]["variable"], values = df_db_plot_true[df_db_plot_true["Check"] == checks[2]]["value"]),2,1 )
fig.add_trace(go.Pie(labels = df_db_plot_false[df_db_plot_false["Check"] == checks[2]]["variable"], values = df_db_plot_false[df_db_plot_false["Check"] == checks[2]]["value"]),2,2 )
fig.add_trace(go.Pie(labels = df_db_plot_true[df_db_plot_true["Check"] == checks[4]]["variable"], values = df_db_plot_true[df_db_plot_true["Check"] == checks[4]]["value"]),3,1 )
fig.add_trace(go.Pie(labels = df_db_plot_false[df_db_plot_false["Check"] == checks[4]]["variable"], values = df_db_plot_false[df_db_plot_false["Check"] == checks[4]]["value"]),3,2 )
for i in range(len(fig.layout.annotations)):
    current = fig.layout.annotations[i]["y"]
    fig.layout.annotations[i].update(y = current + 0.1)
fig.show()

In [ ]:
from plotly.subplots import make_subplots
fig_true = make_subplots(rows=3, cols=1, specs=[[{'type':'domain'}],[{'type':'domain'}],[{'type':'domain'}]],
                   subplot_titles = ["Collection Payment in Last 12 Months","Borrower Delinquency","Loan Payments Missed"])

fig_false = make_subplots(rows=3, cols=1, specs=[[{'type':'domain'}],[{'type':'domain'}],[{'type':'domain'}]],
                   subplot_titles = ["No Collection Payment in Last 12 Months","No Borrower Delinquency","No Loan Payments Missed"])


fig_true.add_trace(go.Pie(labels = df_db_plot_true[df_db_plot_true["Check"] == checks[0]]["variable"], values = df_db_plot_true[df_db_plot_true["Check"] == checks[0]]["value"]),1,1 )
fig_false.add_trace(go.Pie(labels = df_db_plot_false[df_db_plot_false["Check"] == checks[0]]["variable"], values = df_db_plot_false[df_db_plot_false["Check"] == checks[0]]["value"]),1,1)
fig_true.add_trace(go.Pie(labels = df_db_plot_true[df_db_plot_true["Check"] == checks[2]]["variable"], values = df_db_plot_true[df_db_plot_true["Check"] == checks[2]]["value"]),2,1 )
fig_false.add_trace(go.Pie(labels = df_db_plot_false[df_db_plot_false["Check"] == checks[2]]["variable"], values = df_db_plot_false[df_db_plot_false["Check"] == checks[2]]["value"]),2,1)
fig_true.add_trace(go.Pie(labels = df_db_plot_true[df_db_plot_true["Check"] == checks[4]]["variable"], values = df_db_plot_true[df_db_plot_true["Check"] == checks[4]]["value"]),3,1 )
fig_false.add_trace(go.Pie(labels = df_db_plot_false[df_db_plot_false["Check"] == checks[4]]["variable"], values = df_db_plot_false[df_db_plot_false["Check"] == checks[4]]["value"]),3,1)

fig_true.update_annotations(yshift=17)
fig_false.update_annotations(yshift=17)
fig_true.update_layout(title = "Delinquent Customers", width = 1000, height = 500)
fig_false.update_layout(title = "Good Standing Customers", width = 1000, height = 500)

fig_true.show()
fig_true.write_image("true_pie_charts.png")

In [ ]:

fig_false.show()
fig_false.write_image("false_pie_charts.png")

In [ ]:
grouped["id"]

In [ ]:
px.scatter(df.sample(frac = 0.05), x = "loan_amnt", y = "installment", color = "default_ind", opacity = 0.7, facet_row = "grade",
           category_orders = {"grade":["A","B","C","D","E","F","G"]}, height = 1000)

In [ ]:
px.scatter(df.sample(frac = 0.05), x = "loan_amnt", y = "installment", color = "grade", opacity = 0.7, category_orders = {"grade":["A","B","C","D","E","F","G"]})

In [ ]:
grade_list = ["A","B","C","D","E","F","G"]
fig = px.scatter(df.sample(frac = 0.05), x = "loan_amnt", y = "installment", color = "grade", opacity = 0.7, category_orders = {"grade":["A","B","C","D","E","F","G"]}, template = "plotly_white", width = 1000, height = 500)
fig.update_layout(
                 title = "Loan Amount vs Installment Amount",
                 yaxis_title = "Installment (USD)",
                 xaxis_title = "Loan Amount (USD)",
                 legend_title = "Grade", yaxis_range = [0,1500], xaxis_range = [0,36000])
fig.write_image("loan_amnt_installment_grade_overall.png")
fig.show()

In [ ]:
grade_list = ["A","B","C","D","E","F","G"]
sample = df.sample(frac = 0.05)
for grade in grade_list:
    order = [grade+str(x) for x in range(1,6)]
    fig = px.scatter(sample[sample["grade"] == grade], x = "loan_amnt", y = "installment", color = "sub_grade", opacity = 0.7, category_orders = {"sub_grade":order}, template = "plotly_white", width = 1000, height = 500)
    fig.update_layout(
                     title = "Loan Amount vs Installment Amount",
                     yaxis_title = "Installment (USD)",
                     xaxis_title = "Loan Amount (USD)",
                     legend_title = "Grade", yaxis_range = [0,1500], xaxis_range = [0,36000])
    fig.write_image("loan_amnt_installment_subgrade_"+grade+".png")


In [ ]:
df['collections_12_mths_ex_med']

In [ ]:
px.histogram(df[(df["total_rec_late_fee"] > 0) & (df["total_rec_late_fee"] <100)], x = "total_rec_late_fee", color = "default_ind")

In [ ]:
px.histogram(df[(df["total_rec_late_fee"] > 0) & (df["total_rec_late_fee"] <100)], x = "total_rec_late_fee", color = "default_ind", barnorm = 'percent')